# Título   Crear clase ternaria

## Autor: Jose Chelquer
## Fecha de última modificación: 22/10/2025
## Descripción:

Crear clase ternaria a partir del crudo grabando otro dataset

## Parámetros

< Descripción de cada uno de los parámetros que utiliza el job >


In [8]:

usar_gdrive=False    # Poner en true si se va a ejecutar con archivos de google drive

## Input

< Archivos de datos (csv.gz) con sus paths que van a consumirse por el job>

In [9]:
# Ejemplo


dataset_path = '~/buckets/b1/datasets'
dataset_file = 'competencia_02_crudo.csv.gz'

## Output

< Archivos, bases de datos, modelos que va a generar el job>

In [10]:
output_file='competencia_02.csv.gz'


In [11]:
## Procesos

### Paquetes necesarios

## Código del proceso

< Todo el código a partir de aquí debe poder ejecutarse sin necesidad de parametrizar nada>

In [12]:
import os
import pandas as pd

In [13]:
if usar_gdrive:
    from google.colab import drive
    drive.mount('/content/drive')

In [14]:
# leer el dataset

file_path = os.path.join(dataset_path, dataset_file)
print (f'Leyendo {file_path}')
# Read the CSV file into a DataFrame
df = pd.read_csv(file_path)
df.shape

Leyendo ~/buckets/b1/datasets/competencia_02_crudo.csv.gz


(4735593, 154)

In [15]:
# Dado un mes en formato aaaamm y una cantidad de meses
# devuelve el mes a distancia meses de mes
def sig_mes(mes, meses):
  aaaa=mes//100
  mm=mes%100+meses
  aExtra=(mm-1)//12
  mm=mm-12*aExtra
  return (aaaa+aExtra)*100+mm


In [17]:
df["cli_mes"]=df["numero_de_cliente"].astype(str)+df["foto_mes"].astype(str)
df["cli_mesMas1"]=df["numero_de_cliente"].astype(str)+df["foto_mes"].apply(lambda x: sig_mes(x, 1)).astype(str)
df["cli_mesMas2"]=df["numero_de_cliente"].astype(str)+df["foto_mes"].apply(lambda x: sig_mes(x, 2)).astype(str)

In [18]:
print (f'Detectando existe1 y existe2')
df1=df['cli_mes']   #sólo las columnas del cliente
# Realiza la unión entre df y df1
resultado = df.merge(df1, left_on='cli_mesMas1', right_on='cli_mes', how='left', indicator=True)
df['existe1'] = resultado['_merge'] == 'both'
resultado = df.merge(df1, left_on='cli_mesMas2', right_on='cli_mes', how='left', indicator=True)
df['existe2'] = resultado['_merge'] == 'both'


Detectando existe1 y existe2


In [19]:
print (f'Creando clase ternaria')
df[ "clase_ternaria"]="CONTINUA"          #default
df.loc[df["existe2"]==False, "clase_ternaria"]="BAJA+2"
df.loc[df["existe1"]==False, "clase_ternaria"]="BAJA+1"
df.loc[df["existe2"]==True, "clase_ternaria"]="CONTINUA"

Creando clase ternaria


In [20]:
# Eliminar los campos creados
print (f'Eliminando campos creados')
df = df.drop(columns=['cli_mes', 'cli_mesMas1', 'cli_mesMas2', 'existe1', 'existe2'])

Eliminando campos creados


In [ ]:
df['clase_ternaria'].value_counts()

In [ ]:
df.shape[0]

In [ ]:
ruta_nuevo_archivo = os.path.join(dataset_path, output_file)
# Guarda el DataFrame en el nuevo archivo
print (f'Guardando el archivo de salida')
df.to_csv(ruta_nuevo_archivo, index=False, compression='gzip')
print(f"El archivo ha sido guardado en: {ruta_nuevo_archivo}")

Guardando el archivo de salida
